In [1]:
import sys
print(sys.executable)


/opt/miniconda3/bin/python


In [2]:
# Cài đặt các thư viện cần thiết
!pip install -q faiss-cpu sentence-transformers langchain-text-splitters rouge numpy scipy spacy

In [3]:
import os
import json
import numpy as np
import pandas as pd
import faiss
import spacy
import networkx as nx
from datetime import datetime
from dataclasses import dataclass, field
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List, Union
from rouge import Rouge


/opt/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Thử (1)

In [4]:
import os
os.environ["NVAPI_KEY"] = "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"

thử (2) NANO graphRAG

In [5]:
# simple_graphrag.py (upgraded -> v3.0)
import os
import json
import logging
from typing import List, Set, Tuple, Dict, Any, Optional, Callable, Union
from dataclasses import dataclass, field
from datetime import datetime

import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
import spacy
import networkx as nx
from rouge import Rouge
# import google.generativeai as genai  # Gemini SDK
import os
os.environ["NVAPI_KEY"] = "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"
# 3) Create a client wrapper
from openai import OpenAI
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.getenv("NVAPI_KEY")
)

# Optional / extra libs (used if installed)
try:
    import tiktoken
except Exception:
    tiktoken = None

try:
    from node2vec import Node2Vec
except Exception:
    Node2Vec = None

# Optional clustering libs
try:
    import igraph as ig
    import leidenalg
except Exception:
    ig = None
    leidenalg = None

# try:
from networkx.algorithms import community as nx_community
# except Exception:
#     nx_community = None

# ---------- Logging ----------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("SimpleGraphRAG")



# ---------- Utility LLM (Kimi K2) ----------
# async
def llm_generate(prompt: str,
                       model: str = "moonshotai/kimi-k2-instruct-0905",
                       temperature: float = 0.2,
                       max_tokens: int = 1024):
    """
    Gọi Kimi-K2-Instruct qua NVIDIA NIM bằng API OpenAI-compatible.
    """
    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            top_p=0.9,
            max_tokens=max_tokens
        )

        # return resp.choices[0].message["content"].strip()   ==>  sửa từ gemini --> Kimi
        return resp.choices[0].message.content.strip()

    except Exception as e:
        print(f"[LLM ERROR] {e}")
        return ""


# ---------------- Configuration dataclass (Nano-like) ----------------
@dataclass
class GraphRAGConfig:
    working_dir: str = field(
        default_factory=lambda: f"./nano_graphrag_cache_{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"
    )

    # graph mode
    enable_local: bool = True
    enable_naive_rag: bool = False

    # text chunking
    tokenizer_type: str = "tiktoken"  # or 'huggingface' or 'char'
    # tiktoken_model_name: str = "gpt-4o"
    tiktoken_model_name: str = "cl100k_base"

    huggingface_model_name: str = "bert-base-uncased"
    chunk_token_size: int = 1000
    chunk_overlap_token_size: int = 100
    chunk_char_size: int = 1200
    chunk_char_overlap: int = 100
    chunk_func: Optional[Callable] = None  # can be set to a custom chunking function

    # entity extraction
    entity_extract_max_gleaning: int = 1
    entity_summary_to_max_tokens: int = 500

    # graph clustering
    graph_cluster_algorithm: str = "greedy"  # 'leiden' or 'louvain' or 'none'
    max_graph_cluster_size: int = 8
    graph_cluster_seed: int = 0xDEADBEEF

    # node embedding
    node_embedding_algorithm: str = "node2vec"
    node2vec_params: dict = field(
        default_factory=lambda: {
            "dimensions": 128,
            "num_walks": 10,
            "walk_length": 40,
            "window_size": 3,
            "iterations": 3,
            "random_seed": 3,
        }
    )

    # community reports
    special_community_report_llm_kwargs: dict = field(
        default_factory=lambda: {"response_format": {"type": "json_object"}}
    )

    # text embedding (batching)
    embedding_batch_num: int = 32
    embedding_func_max_async: int = 16
    query_better_than_threshold: float = 0.2


# ---------------- Token-based splitter (tiktoken) ----------------
class TokenSplitter:
    def __init__(self, cfg: GraphRAGConfig):
        self.cfg = cfg
        if tiktoken is None:
            raise RuntimeError(
                "tiktoken not installed; install with `pip install tiktoken`"
            )

        # 🔒 Stable tokenizer
        try:
            self.enc = tiktoken.encoding_for_model(cfg.tiktoken_model_name)
        except KeyError:
            logger.warning(
                f"tiktoken cannot map model '{cfg.tiktoken_model_name}'. "
                "Falling back to cl100k_base."
            )
            self.enc = tiktoken.get_encoding("cl100k_base")

        self.chunk_size = cfg.chunk_token_size
        self.overlap = cfg.chunk_overlap_token_size

    def chunk(self, text: str) -> List[str]:
        tokens = self.enc.encode(text)
        chunks: List[str] = []
        step = max(1, self.chunk_size - self.overlap)
        for i in range(0, len(tokens), step):
            sub = tokens[i:i + self.chunk_size]
            if not sub:
                continue
            chunks.append(self.enc.decode(sub))
        return chunks


import re

def split_long_context(text, max_chars=800):
    """
    Chia 1 context dài thành nhiều đoạn nhỏ (~800 ký tự),
    đảm bảo mỗi đoạn vẫn là 1 đoạn văn logic.
    """
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current = ""

    for s in sentences:
        if len(current) + len(s) <= max_chars:
            current += " " + s
        else:
            chunks.append(current.strip())
            current = s

    if current.strip():
        chunks.append(current.strip())

    return chunks

# ---------------- Main SimpleGraphRAG (keeps original API) ----------------
class SimpleGraphRAG:
    def __init__(
        self,
        embedding_model_name: str = "all-MiniLM-L6-v2",
        chunk_size: int = 1200,
        chunk_overlap: int = 100,
        faiss_index_factory: Optional[str] = None,
        cfg: Optional[GraphRAGConfig] = None,
    ):
        # preserve original args
        self.embedding_model_name = embedding_model_name
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.faiss_index_factory = faiss_index_factory

        # config (nano-like)
        self.cfg = cfg or GraphRAGConfig()
        os.makedirs(self.cfg.working_dir, exist_ok=True)

        # core components
        self.dataset: List[str] = []
        self.chunk_entities: List[Set[str]] = []
        self.nlp = None
        self.embedding_model = None
        self.embeddings: Optional[np.ndarray] = None
        self.index: Optional[faiss.Index] = None
        self.kg: Optional[nx.DiGraph] = None
        self.node_embeddings: Optional[Dict[str, np.ndarray]] = None

        # initialize
        self._init_models()

    def _init_models(self):
        logger.info("Loading embedding model...")
        self.embedding_model = SentenceTransformer(self.embedding_model_name)

        logger.info("Loading spaCy model...")
        try:
            self.nlp = spacy.load("en_core_web_sm")
            self.nlp.max_length = 10_000_000
        except OSError:
            raise RuntimeError("Chưa có model spaCy. Chạy: python -m spacy download en_core_web_sm")

    # ---------------- Load + Chunk ----------------
    def load_json_and_concat(self, json_path: str, text_key: str = "context") -> str:
        logger.info(f"Loading JSON from {json_path} ...")
        try:
            df = pd.read_json(json_path)
            if text_key not in df.columns:
                raise ValueError(f"Key '{text_key}' not found. Columns: {df.columns.tolist()}")
            full_text = " ".join(df[text_key].astype(str).tolist())
        except Exception as e:
            logger.warning(f"Lỗi đọc JSON: {e}. Dùng fallback text.")
            full_text = (
                "Basal cell carcinoma (BCC) is the most common type of skin cancer. "
                "It is frequently treated with surgery. Fair skin increases the risk."
            )
        return full_text

    # import re

    # def split_long_context(text, max_chars=1000):
    #     sentences = re.split(r'(?<=[.!?])\s+', text)
    #     chunks = []
    #     current = ""

    #     for s in sentences:
    #         if len(current) + len(s) < max_chars:
    #             current += " " + s
    #         else:
    #             chunks.append(current.strip())
    #             current = s

    #     if current:
    #         chunks.append(current.strip())
    #     return chunks

    def chunk_text(self, text: str, method: str = "auto"):
        """
        method: 'auto' -> choose based on cfg.tokenizer_type
                'token' -> tiktoken
                'char' -> RecursiveCharacterTextSplitter
        """
        logger.info("Chunking text ...")
        # use custom chunk func if provided
        if self.cfg.chunk_func is not None:
            logger.info("Using custom chunk_func from cfg")
            chunks = self.cfg.chunk_func(text)
            self.dataset = [c["text"] if isinstance(c, dict) and "text" in c else str(c) for c in chunks]
            logger.info(f"Created {len(self.dataset)} chunks (custom function)")
            return

        mode = method
        if method == "auto":
            mode = "token" if self.cfg.tokenizer_type == "tiktoken" and tiktoken is not None else "char"

        if mode == "token":
            if tiktoken is None:
                logger.warning("tiktoken not available; falling back to char splitter")
                self.dataset = char_splitter(text, self.cfg.chunk_char_size, self.cfg.chunk_char_overlap)
            else:
                splitter = TokenSplitter(self.cfg)
                self.dataset = splitter.chunk(text)
        else:
            self.dataset = char_splitter(text, self.cfg.chunk_char_size, self.cfg.chunk_char_overlap)

        logger.info(f"Created {len(self.dataset)} chunks (mode={mode})")

    # ---------------- Embeddings + FAISS ----------------
    def build_embeddings_and_index(self, normalize: bool = True, batch_size: Optional[int] = None):
        if not self.dataset:
            raise RuntimeError("Dataset empty. Run chunk_text() first.")
        batch_size = batch_size or self.cfg.embedding_batch_num
        logger.info("Computing embeddings ...")
        emb_list = []
        for i in range(0, len(self.dataset), batch_size):
            batch = self.dataset[i:i + batch_size]
            emb_batch = self.embedding_model.encode(batch, normalize_embeddings=normalize)
            emb_list.append(emb_batch)
        emb = np.vstack(emb_list).astype("float32")
        # extra safe-normalize
        norms = np.linalg.norm(emb, axis=1, keepdims=True)
        norms[norms == 0.0] = 1.0
        emb = emb / norms
        self.embeddings = emb

        d = emb.shape[1]
        logger.info(f"Creating FAISS index (dimension={d})")
        if self.faiss_index_factory:
            try:
                self.index = faiss.index_factory(d, self.faiss_index_factory)
            except Exception as e:
                logger.warning(f"faiss.index_factory failed: {e}, falling back to IndexFlatIP")
                self.index = faiss.IndexFlatIP(d)
        else:
            self.index = faiss.IndexFlatIP(d)
        self.index.add(emb)
        logger.info("FAISS index built")

        # cache embeddings
        try:
            np.save(os.path.join(self.cfg.working_dir, "embeddings.npy"), self.embeddings)
            logger.info(f"Saved embeddings to {self.cfg.working_dir}/embeddings.npy")
        except Exception:
            pass

    
    def build_kg(self, use_dependency: bool = True):
        """
        Build a Knowledge Graph using chunk-based NER (safe for large datasets).
        Instead of processing full_text (which can exceed spaCy limits),
        we run spaCy only on each chunk.
        """
        logger.info("Building Knowledge Graph (chunk-based NER)...")

        self.kg = nx.DiGraph()
        entity_nodes = set()
        self.chunk_entities = []

        # --- NER + relation extraction per chunk ---
        for chunk in self.dataset:
            cdoc = self.nlp(chunk)

            # entities in this chunk
            # ents = [e.text.strip() for e in cdoc.ents if e.text.strip()]
            # --- robust entity extraction (medical-safe) ---
            ents = set()

            # 1️⃣ spaCy NER (nếu có)
            for ent in cdoc.ents:
                ents.add(ent.text.strip())

            # 2️⃣ fallback: noun chunks (QUAN TRỌNG)
            for np in cdoc.noun_chunks:
                text = np.text.strip().lower()
                if len(text.split()) >= 2:          # bỏ noun quá ngắn
                    ents.add(text)

            ents = list(ents)

            ent_set = set(ents)
            self.chunk_entities.append(ent_set)

            # add entity nodes
            for e in ent_set:
                if e not in self.kg:
                    # use entity label if available (PERSON, ORG, etc)
                    label = None
                    for ent in cdoc.ents:
                        if ent.text.strip() == e:
                            label = ent.label_
                            break
                    self.kg.add_node(e, label=label or "UNKNOWN")
                entity_nodes.add(e)

            # extract relations inside this chunk
            for sent in cdoc.sents:
                sent_ents = [e.text.strip() for e in sent.ents]

                # --- dependency-based relation extraction ---
                if use_dependency:
                    subj = None
                    obj = None
                    verb = None
                    for token in sent:
                        if "subj" in token.dep_:
                            subj = token.text
                            verb = token.head.lemma_
                            for child in token.head.children:
                                if "obj" in child.dep_:
                                    obj = child.text
                                    if subj in entity_nodes and obj in entity_nodes:
                                        self.kg.add_edge(subj, obj, relation=verb)

                # --- fallback: co-occurrence graph ---
                for i in range(len(sent_ents)):
                    for j in range(i + 1, len(sent_ents)):
                        a, b = sent_ents[i], sent_ents[j]
                        if a != b and not self.kg.has_edge(a, b):
                            self.kg.add_edge(a, b, relation="co_occurs_with")

        logger.info(f"KG built with {self.kg.number_of_nodes()} nodes, {self.kg.number_of_edges()} edges")


    # ---------------- Graph clustering (optional) ----------------
    def cluster_graph(self):
        
        
        algo = self.cfg.graph_cluster_algorithm.lower()
        if algo == "none":
            logger.info("Graph clustering disabled (algorithm=none).")
            return None

        # --- Leiden via igraph (optional) ---
        try:
            import igraph as ig
            import leidenalg
            if algo == "leiden":
                logger.info("Running Leiden clustering via igraph + leidenalg.")
                g = ig.Graph.TupleList(self.kg.edges(), directed=True)
                partition = leidenalg.find_partition(g, leidenalg.RBConfigurationVertexPartition)
                clusters = [list(map(str, com)) for com in partition]
                logger.info(f"Leiden found {len(clusters)} clusters.")
                return clusters
        except ImportError:
            logger.warning("igraph or leidenalg not installed; skipping Leiden clustering.")

        # --- NetworkX community fallback ---
        if nx_community is not None and algo in ("louvain", "greedy", "labelprop"):
            logger.info("Running NetworkX community clustering (fallback).")
            communities = nx_community.greedy_modularity_communities(self.kg.to_undirected())
            clusters = [list(c) for c in communities]
            logger.info(f"NetworkX found {len(clusters)} communities.")
            return clusters

        logger.warning("No clustering library available or unsupported algorithm; skipping clustering.")
        return None

    # ---------------- Node embeddings (node2vec) ----------------
    def embed_graph_nodes(self):
        if Node2Vec is None:
            logger.warning("node2vec not installed; skipping node embedding.")
            return None
        if self.kg is None or self.kg.number_of_nodes() == 0:
            logger.warning("KG empty; run build_kg() first.")
            return None
        params = self.cfg.node2vec_params
        logger.info("Running Node2Vec to generate node embeddings...")
        # node2vec expects undirected or directed networkx graph
        node2vec = Node2Vec(self.kg, dimensions=params.get("dimensions", 128),
                            walk_length=params.get("walk_length", 40),
                            num_walks=params.get("num_walks", 10),
                            workers=2)
        model = node2vec.fit(window=params.get("window_size", 3), min_count=1, batch_words=4)
        # store embeddings in dict
        self.node_embeddings = {node: model.wv[str(node)] for node in self.kg.nodes()}
        logger.info(f"Generated node embeddings for {len(self.node_embeddings)} nodes.")
        # optional cache
        try:
            np.save(os.path.join(self.cfg.working_dir, "node_embeddings.npy"), self.node_embeddings)
        except Exception:
            pass
        return self.node_embeddings

    # ---------------- GraphRAG Query (hybrid) ----------------
    def _semantic_search(self, query: str, top_k: int = 5):
        q_emb = self.embedding_model.encode([query], normalize_embeddings=True).astype("float32")
        D, I = self.index.search(q_emb, top_k)
        return [(int(i), float(D[0][idx])) for idx, i in enumerate(I[0])]

    def _graph_search_scores(self, query: str):
        q_doc = self.nlp(query)
        q_entities = set([ent.text.strip() for ent in q_doc.ents if ent.text.strip()])
        if len(q_entities) == 0:
            return np.zeros(len(self.dataset), dtype=float)
        scores = np.array([len(self.chunk_entities[i].intersection(q_entities)) for i in range(len(self.dataset))], dtype=float)
        if scores.max() > 0:
            scores = scores / (scores.max() + 1e-12)
        return scores

    def graphrag_query(self, query: str, top_k: int = 5, alpha: float = 0.8):
        """
        Combine semantic + graph signals.
        alpha: weight for semantic retrieval (0..1). final_score = alpha*sem + (1-alpha)*graph
        """
        # semantic candidates
        sem = self._semantic_search(query, top_k=top_k * 2)  # retrieve more to allow reranking
        sem_idxs = [i for i, s in sem]
        sem_scores = {i: s for (i, s) in sem}

        # graph scores across dataset
        graph_scores = self._graph_search_scores(query)

        # compute combined score for sem candidates
        combined = []
        for i, sem_score in sem:
            gscore = graph_scores[i] if i < len(graph_scores) else 0.0
            score = alpha * sem_score + (1.0 - alpha) * gscore
            combined.append((i, score))

        # sort and pick top_k
        combined_sorted = sorted(combined, key=lambda x: x[1], reverse=True)[:top_k]
        combined_idxs = [i for i, s in combined_sorted]
        combined_chunks = [self.dataset[i] for i in combined_idxs]

        # gather also top graph-only chunks (if graph points to others not in sem candidates)
        top_graph_idx = np.argsort(graph_scores)[-top_k:][::-1]
        graph_chunks = [self.dataset[i] for i in top_graph_idx if graph_scores[i] > 0 and i not in combined_idxs]

        # KG facts extraction (neighbours of query entities)
        kg_facts = []
        q_doc = self.nlp(query)
        q_entities = set([ent.text.strip() for ent in q_doc.ents if ent.text.strip()])
        for qe in q_entities:
            if self.kg is not None and qe in self.kg:
                # successors & predecessors (limit to a few)
                for nb in list(self.kg.successors(qe))[:3]:
                    rel = self.kg.get_edge_data(qe, nb).get("relation", "related_to")
                    kg_facts.append(f"{qe} {rel} {nb}.")
                for nb in list(self.kg.predecessors(qe))[:3]:
                    rel = self.kg.get_edge_data(nb, qe).get("relation", "related_to")
                    kg_facts.append(f"{nb} {rel} {qe}.")

        combined_context = " ".join(combined_chunks + graph_chunks + kg_facts)
        # combined_context = " ".join(combined_chunks[:3] + kg_facts[:4])
            
        return {"query": query, "context": combined_context, "sem_idxs": combined_idxs}

    # ---------------- LLM Answer ----------------
    def answer_question_llm(self, query: str, llm_model: str = "moonshotai/kimi-k2-instruct-0905", top_k: int = 5, alpha: float = 0.8):
        result = self.graphrag_query(query, top_k=top_k, alpha=alpha)
        context_text = result["context"]
        prompt = f"""
You are a high-accuracy QA assistant. 

You MUST answer using ONLY the provided context.

Your output must follow these rules:

-------------------------------------------------
DETECTION RULES (auto-detect the question type):
- If the question asks for a specific name, place, person, object, event,
  → classify as: FACT RETRIEVAL.
- If the question asks “why”, “how”, “what happened”, “what was the relationship”,
  or requires combining multiple clues,
  → classify as: COMPLEX REASONING.
-------------------------------------------------

ANSWER STYLE RULES:
1. FACT RETRIEVAL:
   - Answer in **3–12 words**.
   - Must be short, factual, and contain ONLY the requested entity.
   - Do NOT add explanations or context.

2. COMPLEX REASONING:
   - Answer in 1–2 concise sentences (12–25 words total).
   - Combine only the essential evidence from the context.
   - No paraphrasing of long passages; no extra narrative.
   - The answer must be strictly grounded in the provided context.

3. If the question_type is "Contextual Summarize":
   - Produce 2–4 sentences.
   - Length must be 80–200 words.
   - Provide a smooth, coherent summary combining *all* major points from context.
   - Use descriptive but concise language.
   - Do NOT shorten aggressively.
   - Do NOT reduce to a single-sentence summary.
   - This rule overrides any sentence-limit rules.

4. If the question_type is "Creative Generation":
   - Follow the exact requested format (diary entry, newspaper article, retelling, etc.).
   - Stay grounded in the given evidence; do not introduce new events or characters.
   - 3–6 sentences depending on style, expressive but controlled.

5. GENERAL RULES:
    - Do NOT add information not present in the context.
    - If context is incomplete, give the best-supported answer.
    - If no relevant information exists, reply EXACTLY:
    no relevant information found

-------------------------------------------------
CONTEXT:
{context_text}

QUESTION:
{query}

ANSWER:
"""
        # fix: chạy async function trong sync
        # import asyncio
        # answer = asyncio.run(llm_generate(prompt, model=llm_model))
        answer = llm_generate(prompt, model=llm_model)

        return answer, result
       





    # ---------------- Evaluation ----------------
    def evaluate_answer(self, pred: str, truth: str) -> float:
        scorer = Rouge()
        scores = scorer.get_scores(pred, truth)
        return scores[0]["rouge-l"]["f"]

    # ---------------- Cache helpers ----------------
    def save_index(self, fname: str = "faiss.index"):
        if self.index is None:
            raise ValueError("No index to save.")
        path = os.path.join(self.cfg.working_dir, fname)
        faiss.write_index(self.index, path)
        logger.info(f"FAISS index written to {path}")

    def load_index(self, fname: str = "faiss.index"):
        path = os.path.join(self.cfg.working_dir, fname)
        if not os.path.exists(path):
            raise FileNotFoundError(path)
        self.index = faiss.read_index(path)
        logger.info(f"FAISS index loaded from {path}")

    def save_embeddings(self, fname: str = "embeddings.npy"):
        if self.embeddings is None:
            raise ValueError("No embeddings to save.")
        np.save(os.path.join(self.cfg.working_dir, fname), self.embeddings)
        logger.info("Embeddings saved.")

    def load_embeddings(self, fname: str = "embeddings.npy"):
        path = os.path.join(self.cfg.working_dir, fname)
        if not os.path.exists(path):
            raise FileNotFoundError(path)
        self.embeddings = np.load(path)
        logger.info("Embeddings loaded.")

    # ---------------- Convenience run -> inference on JSON questions ----------------
def run_inference_on_medical_json(rag, json_path, num_questions=10,question_type_filter="Fact Retrieval"):
    """
    Chạy LLM trên các câu hỏi JSON được lọc theo question_type.
    Mặc định chạy các câu 'Fact Retrieval'.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    # Lọc các câu hỏi theo loại yêu cầu
    # Lọc các câu hỏi theo loại yêu cầu
    filtered = [
        item for item in data
        if item.get("question_type", "").lower() == question_type_filter.lower()
    ]

    if not filtered:
        print(f"❌ Không tìm thấy câu hỏi loại '{question_type_filter}' trong file: {json_path}")
        return

    print(f"🔹 Found {len(filtered)} questions of type '{question_type_filter}'")
    print(f"➡ Running inference on first {num_questions}...\n")

    # print(f"🔹 Running inference on {num_questions} questions from {json_path}\n")

    # ❗ USE filtered list, NOT full dataset
    for i, item in enumerate(filtered[:num_questions]):
        question = item.get("question", "") or item.get("q", "")
        ground_truth = item.get("answer", "") or item.get("a", "")
        # question_type = item.get("question_type", "")
        # gọi LLM
        ans, _ = rag.answer_question_llm(question)

        # xử lý câu trả lời
        sentences = [s.strip() for s in ans.replace("\n", " ").split(".") if s.strip()]
        short_ans = ". ".join(sentences[:5]).strip()
        if short_ans and not short_ans.endswith("."):
            short_ans += "."

        # in kết quả
        print(f"=== Q{i+1}: {question}")
        print(f"Kimi K2 Answer:\n{short_ans}\n")
        print(f"Ground Truth:\n{ground_truth}\n")
        print("-" * 120)

def load_txt(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()


# ---------------- Example usage ----------------
if __name__ == "__main__":
    # create rag with default config (you can pass a GraphRAGConfig instance to override)
    rag = SimpleGraphRAG(embedding_model_name="all-MiniLM-L6-v2", chunk_size=1200, chunk_overlap=100)

    
    # 1) load corpus# Paths (change to your local dataset)
    # corpus_path = "/KGC/SAT/aligner/data/FB15k-237N/id2text.txt"
    corpus_path = "SAT/aligner/data/FB15k-237N/id3text.txt"


    # questions_path = "/Users/duyanhle1501/Duy Anh Le Code/GraphRAG-Benchmark/Datasets/Questions/medical_questions.json"

    # text = rag.load_json_and_concat(corpus_path)
    text = load_txt(corpus_path)

    rag.dataset = split_long_context(text, max_chars=800)

    # 2) chunk (uses config tokenizer; default is tiktoken if available)
    # rag.chunk_text(text, method="auto")

    # 3) embedding + index
    rag.build_embeddings_and_index()

    # 4) build a proper KG (NER + simple dependency relations)
    rag.build_kg(use_dependency=True)

    import pickle
    import os

    os.makedirs("sat_data", exist_ok=True)

    with open("sat_data/kg.pkl", "wb") as f:
        pickle.dump(rag.kg, f)

    print("✅ Saved KG to sat_data/kg.pkl")
    print("Nodes:", rag.kg.number_of_nodes())
    print("Edges:", rag.kg.number_of_edges())


    # 5) optional: cluster graph & embed nodes
    # clusters = rag.cluster_graph()
    # node_embs = rag.embed_graph_nodes()

    # 6) run inference on first 5 questions and print top 5-sentence summary
    # run_inference_on_medical_json(rag, questions_path, num_questions=10, question_type_filter="Fact Retrieval")

    # ====== SAVE CONTEXTS FOR PART B ======
    # import json

    # contexts = []

    # with open(questions_path, "r", encoding="utf-8") as f:
    #     data = json.load(f)

    # filtered = [
    #     item for item in data
    #     if item.get("question_type", "").lower() == "fact retrieval"
    # ]

    # for item in filtered[:10]:
    #     q = item["question"]
    #     r = rag.graphrag_query(q)
    #     contexts.append({
    #         "question": q,
    #         "context": r["context"]
    #     })

    # with open("contexts.json", "w", encoding="utf-8") as f:
    #     json.dump(contexts, f, indent=2, ensure_ascii=False)

    # print("✅ contexts.json saved (Part A done)")





INFO:SimpleGraphRAG:Loading embedding model...
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:SimpleGraphRAG:Loading spaCy model...
INFO:SimpleGraphRAG:Computing embeddings ...
Batches: 100%|██████████| 1/1 [00:00<00:00, 14.07it/s]
INFO:SimpleGraphRAG:Creating FAISS index (dimension=384)
INFO:SimpleGraphRAG:FAISS index built
INFO:SimpleGraphRAG:Saved embeddings to ./nano_graphrag_cache_2026-01-20-10-37-23/embeddings.npy
INFO:SimpleGraphRAG:Building Knowledge Graph (chunk-based NER)...
INFO:SimpleGraphRAG:KG built with 5852 nodes, 8579 edges


✅ Saved KG to sat_data/kg.pkl
Nodes: 5852
Edges: 8579


In [7]:
text = rag.load_json_and_concat(corpus_path)
doc = rag.nlp(text[:1000])

print([(ent.text, ent.label_) for ent in doc.ents])


INFO:SimpleGraphRAG:Loading JSON from /Users/duyanhle1501/Duy Anh Le Code/GraphRAG-Benchmark/Datasets/Corpus/medical.json ...


[('BCC', 'ORG')]


Thêm QWEN NÈ


In [5]:
pip install peft


Note: you may need to restart the kernel to use updated packages.


In [7]:
# !pip install ipykernel
# !python -m ipykernel install --user --name kgc-venv --display-name "KGC (.venv)"


In [ ]:
# # simple_graphrag.py (upgraded -> v3.0)
# import os
# import json
# import logging
# from typing import List, Set, Tuple, Dict, Any, Optional, Callable, Union
# from dataclasses import dataclass, field
# from datetime import datetime

# import numpy as np
# import pandas as pd
# import faiss
# from sentence_transformers import SentenceTransformer
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# import spacy
# import networkx as nx
# from rouge import Rouge
# import os

# # ---------- Local Qwen (SAT LoRA) ----------
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import PeftModel

# QWEN_MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
# SAT_LORA_PATH = "sat_lora_model"

# def load_qwen_sat():
#     print("🔹 Loading Qwen + SAT LoRA (CPU only, safe)")

#     device = "cpu"
#     dtype = torch.float32   # BẮT BUỘC

#     tokenizer = AutoTokenizer.from_pretrained(QWEN_MODEL_NAME)

#     base_model = AutoModelForCausalLM.from_pretrained(
#         QWEN_MODEL_NAME,
#         dtype=dtype,
#         device_map=None,
#         low_cpu_mem_usage=True
#     )

#     model = PeftModel.from_pretrained(base_model, SAT_LORA_PATH)
#     model.eval()

#     return tokenizer, model, device

# # QWEN_TOKENIZER, QWEN_MODEL, QWEN_DEVICE = load_qwen_sat()
# # QWEN_TOKENIZER = None
# # QWEN_MODEL = None
# # QWEN_DEVICE = None



# def qwen_reasoning(query: str, context: str, max_tokens: int = 128) -> str:
#     global QWEN_TOKENIZER, QWEN_MODEL, QWEN_DEVICE

#     if QWEN_MODEL is None:
#         QWEN_TOKENIZER, QWEN_MODEL, QWEN_DEVICE = load_qwen_sat()

#     prompt = f"""
# You are a reasoning assistant trained on SAT-style logic.

# Context:
# {context}

# Question:
# {query}

# Reasoning:
# """

#     inputs = QWEN_TOKENIZER(prompt, return_tensors="pt")

#     with torch.no_grad():
#         outputs = QWEN_MODEL.generate(
#             **inputs,
#             max_new_tokens=max_tokens,
#             do_sample=False
#         )

#     text = QWEN_TOKENIZER.decode(outputs[0], skip_special_tokens=True)
#     return text.split("Reasoning:")[-1].strip()

# # ==============================
# # 🔹 Kimi K2 (Remote via NVIDIA NIM)
# # ==============================

# os.environ["NVAPI_KEY"] = "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"
# # 3) Create a client wrapper
# from openai import OpenAI
# client = OpenAI(
#     base_url="https://integrate.api.nvidia.com/v1",
#     api_key=os.getenv("NVAPI_KEY")
# )

# # Optional / extra libs (used if installed)
# try:
#     import tiktoken
# except Exception:
#     tiktoken = None

# try:
#     from node2vec import Node2Vec
# except Exception:
#     Node2Vec = None

# # Optional clustering libs
# try:
#     import igraph as ig
#     import leidenalg
# except Exception:
#     ig = None
#     leidenalg = None

# # try:
# from networkx.algorithms import community as nx_community
# # except Exception:
# #     nx_community = None

# # ---------- Logging ----------
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger("SimpleGraphRAG")



# # ---------- Utility LLM (Kimi K2) ----------
# # async
# def llm_generate(prompt: str,
#                        model: str = "moonshotai/kimi-k2-instruct-0905",
#                        temperature: float = 0.2,
#                        max_tokens: int = 1024):
#     """
#     Gọi Kimi-K2-Instruct qua NVIDIA NIM bằng API OpenAI-compatible.
#     """
#     try:
#         resp = client.chat.completions.create(
#             model=model,
#             messages=[{"role": "user", "content": prompt}],
#             temperature=temperature,
#             top_p=0.9,
#             max_tokens=max_tokens
#         )

#         # return resp.choices[0].message["content"].strip()   ==>  sửa từ gemini --> Kimi
#         return resp.choices[0].message.content.strip()

#     except Exception as e:
#         print(f"[LLM ERROR] {e}")
#         return ""


# # ---------------- Configuration dataclass (Nano-like) ----------------
# @dataclass
# class GraphRAGConfig:
#     working_dir: str = field(
#         default_factory=lambda: f"./nano_graphrag_cache_{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"
#     )

#     # graph mode
#     enable_local: bool = True
#     enable_naive_rag: bool = False

#     # text chunking
#     tokenizer_type: str = "tiktoken"  # or 'huggingface' or 'char'
#     # tiktoken_model_name: str = "gpt-4o"
#     tiktoken_model_name: str = "cl100k_base"

#     huggingface_model_name: str = "bert-base-uncased"
#     chunk_token_size: int = 1000
#     chunk_overlap_token_size: int = 100
#     chunk_char_size: int = 1200
#     chunk_char_overlap: int = 100
#     chunk_func: Optional[Callable] = None  # can be set to a custom chunking function

#     # entity extraction
#     entity_extract_max_gleaning: int = 1
#     entity_summary_to_max_tokens: int = 500

#     # graph clustering
#     graph_cluster_algorithm: str = "greedy"  # 'leiden' or 'louvain' or 'none'
#     max_graph_cluster_size: int = 8
#     graph_cluster_seed: int = 0xDEADBEEF

#     # node embedding
#     node_embedding_algorithm: str = "node2vec"
#     node2vec_params: dict = field(
#         default_factory=lambda: {
#             "dimensions": 128,
#             "num_walks": 10,
#             "walk_length": 40,
#             "window_size": 3,
#             "iterations": 3,
#             "random_seed": 3,
#         }
#     )

#     # community reports
#     special_community_report_llm_kwargs: dict = field(
#         default_factory=lambda: {"response_format": {"type": "json_object"}}
#     )

#     # text embedding (batching)
#     embedding_batch_num: int = 32
#     embedding_func_max_async: int = 16
#     query_better_than_threshold: float = 0.2


# # ---------------- Token-based splitter (tiktoken) ----------------
# class TokenSplitter:
#     def __init__(self, cfg: GraphRAGConfig):
#         self.cfg = cfg
#         if tiktoken is None:
#             raise RuntimeError(
#                 "tiktoken not installed; install with `pip install tiktoken`"
#             )

#         # 🔒 Stable tokenizer
#         try:
#             self.enc = tiktoken.encoding_for_model(cfg.tiktoken_model_name)
#         except KeyError:
#             logger.warning(
#                 f"tiktoken cannot map model '{cfg.tiktoken_model_name}'. "
#                 "Falling back to cl100k_base."
#             )
#             self.enc = tiktoken.get_encoding("cl100k_base")

#         self.chunk_size = cfg.chunk_token_size
#         self.overlap = cfg.chunk_overlap_token_size

#     def chunk(self, text: str) -> List[str]:
#         tokens = self.enc.encode(text)
#         chunks: List[str] = []
#         step = max(1, self.chunk_size - self.overlap)
#         for i in range(0, len(tokens), step):
#             sub = tokens[i:i + self.chunk_size]
#             if not sub:
#                 continue
#             chunks.append(self.enc.decode(sub))
#         return chunks
    
# def char_splitter(text: str, size: int, overlap: int) -> List[str]:
#         splitter = RecursiveCharacterTextSplitter(
#             chunk_size=size,
#             chunk_overlap=overlap,
#             separators=["\n\n", "\n", ".", " ", ""],
#         )
#         return [d.page_content for d in splitter.create_documents([text])]

# # ==============================
# # 🔹 SimpleGraphRAG
# # ==============================
# class SimpleGraphRAG:
#     def __init__(
#         self,
#         embedding_model_name: str = "all-MiniLM-L6-v2",
#         chunk_size: int = 1200,
#         chunk_overlap: int = 100,
#         faiss_index_factory: Optional[str] = None,
#         cfg: Optional[GraphRAGConfig] = None,
#     ):
#         # preserve original args
#         self.embedding_model_name = embedding_model_name
#         self.chunk_size = chunk_size
#         self.chunk_overlap = chunk_overlap
#         self.faiss_index_factory = faiss_index_factory

#         # config (nano-like)
#         self.cfg = cfg or GraphRAGConfig()
#         os.makedirs(self.cfg.working_dir, exist_ok=True)

#         # core components
#         self.dataset: List[str] = []
#         self.chunk_entities: List[Set[str]] = []
#         self.nlp = None
#         self.embedding_model = None
#         self.embeddings: Optional[np.ndarray] = None
#         self.index: Optional[faiss.Index] = None
#         self.kg: Optional[nx.DiGraph] = None
#         self.node_embeddings: Optional[Dict[str, np.ndarray]] = None

#         # initialize
#         self._init_models()

#     def _init_models(self):
#         logger.info("Loading embedding model...")
#         self.embedding_model = SentenceTransformer(self.embedding_model_name)

#         logger.info("Loading spaCy model...")
#         try:
#             self.nlp = spacy.load("en_core_web_sm")
#             self.nlp.max_length = 10_000_000
#         except OSError:
#             raise RuntimeError("Chưa có model spaCy. Chạy: python -m spacy download en_core_web_sm")

#     # ---------------- Load + Chunk ----------------
#     def load_json_and_concat(self, json_path: str, text_key: str = "context") -> str:
#         logger.info(f"Loading JSON from {json_path} ...")
#         try:
#             df = pd.read_json(json_path)
#             if text_key not in df.columns:
#                 raise ValueError(f"Key '{text_key}' not found. Columns: {df.columns.tolist()}")
#             full_text = " ".join(df[text_key].astype(str).tolist())
#         except Exception as e:
#             logger.warning(f"Lỗi đọc JSON: {e}. Dùng fallback text.")
#             full_text = (
#                 "Basal cell carcinoma (BCC) is the most common type of skin cancer. "
#                 "It is frequently treated with surgery. Fair skin increases the risk."
#             )
#         return full_text

#     def chunk_text(self, text: str, method: str = "auto"):
#         """
#         method: 'auto' -> choose based on cfg.tokenizer_type
#                 'token' -> tiktoken
#                 'char' -> RecursiveCharacterTextSplitter
#         """
#         logger.info("Chunking text ...")
#         # use custom chunk func if provided
#         if self.cfg.chunk_func is not None:
#             logger.info("Using custom chunk_func from cfg")
#             chunks = self.cfg.chunk_func(text)
#             self.dataset = [c["text"] if isinstance(c, dict) and "text" in c else str(c) for c in chunks]
#             logger.info(f"Created {len(self.dataset)} chunks (custom function)")
#             return

#         mode = method
#         if method == "auto":
#             mode = "token" if self.cfg.tokenizer_type == "tiktoken" and tiktoken is not None else "char"

#         if mode == "token":
#             if tiktoken is None:
#                 logger.warning("tiktoken not available; falling back to char splitter")
#                 self.dataset = char_splitter(text, self.cfg.chunk_char_size, self.cfg.chunk_char_overlap)
#             else:
#                 splitter = TokenSplitter(self.cfg)
#                 self.dataset = splitter.chunk(text)
#         else:
#             self.dataset = char_splitter(text, self.cfg.chunk_char_size, self.cfg.chunk_char_overlap)

#         logger.info(f"Created {len(self.dataset)} chunks (mode={mode})")

#     # ---------------- Embeddings + FAISS ----------------
#     def build_embeddings_and_index(self, normalize: bool = True, batch_size: Optional[int] = None):
#         if not self.dataset:
#             raise RuntimeError("Dataset empty. Run chunk_text() first.")
#         batch_size = batch_size or self.cfg.embedding_batch_num
#         logger.info("Computing embeddings ...")
#         emb_list = []
#         for i in range(0, len(self.dataset), batch_size):
#             batch = self.dataset[i:i + batch_size]
#             emb_batch = self.embedding_model.encode(batch, normalize_embeddings=normalize)
#             emb_list.append(emb_batch)
#         emb = np.vstack(emb_list).astype("float32")
#         # extra safe-normalize
#         norms = np.linalg.norm(emb, axis=1, keepdims=True)
#         norms[norms == 0.0] = 1.0
#         emb = emb / norms
#         self.embeddings = emb

#         d = emb.shape[1]
#         logger.info(f"Creating FAISS index (dimension={d})")
#         if self.faiss_index_factory:
#             try:
#                 self.index = faiss.index_factory(d, self.faiss_index_factory)
#             except Exception as e:
#                 logger.warning(f"faiss.index_factory failed: {e}, falling back to IndexFlatIP")
#                 self.index = faiss.IndexFlatIP(d)
#         else:
#             self.index = faiss.IndexFlatIP(d)
#         self.index.add(emb)
#         logger.info("FAISS index built")

#         # cache embeddings
#         try:
#             np.save(os.path.join(self.cfg.working_dir, "embeddings.npy"), self.embeddings)
#             logger.info(f"Saved embeddings to {self.cfg.working_dir}/embeddings.npy")
#         except Exception:
#             pass

    
#     def build_kg(self, full_text: str, use_dependency: bool = True):
#         """
#         Build a Knowledge Graph using chunk-based NER (safe for large datasets).
#         Instead of processing full_text (which can exceed spaCy limits),
#         we run spaCy only on each chunk.
#         """
#         logger.info("Building Knowledge Graph (chunk-based NER)...")

#         self.kg = nx.DiGraph()
#         entity_nodes = set()
#         self.chunk_entities = []

#         # --- NER + relation extraction per chunk ---
#         for chunk in self.dataset:
#             cdoc = self.nlp(chunk)

#             # entities in this chunk
#             ents = [e.text.strip() for e in cdoc.ents if e.text.strip()]
#             ent_set = set(ents)
#             self.chunk_entities.append(ent_set)

#             # add entity nodes
#             for e in ent_set:
#                 if e not in self.kg:
#                     # use entity label if available (PERSON, ORG, etc)
#                     label = None
#                     for ent in cdoc.ents:
#                         if ent.text.strip() == e:
#                             label = ent.label_
#                             break
#                     self.kg.add_node(e, label=label or "UNKNOWN")
#                 entity_nodes.add(e)

#             # extract relations inside this chunk
#             for sent in cdoc.sents:
#                 sent_ents = [e.text.strip() for e in sent.ents]

#                 # --- dependency-based relation extraction ---
#                 if use_dependency:
#                     subj = None
#                     obj = None
#                     verb = None
#                     for token in sent:
#                         if "subj" in token.dep_:
#                             subj = token.text
#                             verb = token.head.lemma_
#                             for child in token.head.children:
#                                 if "obj" in child.dep_:
#                                     obj = child.text
#                                     if subj in entity_nodes and obj in entity_nodes:
#                                         self.kg.add_edge(subj, obj, relation=verb)

#                 # --- fallback: co-occurrence graph ---
#                 for i in range(len(sent_ents)):
#                     for j in range(i + 1, len(sent_ents)):
#                         a, b = sent_ents[i], sent_ents[j]
#                         if a != b and not self.kg.has_edge(a, b):
#                             self.kg.add_edge(a, b, relation="co_occurs_with")

#         logger.info(f"KG built with {self.kg.number_of_nodes()} nodes, {self.kg.number_of_edges()} edges")


#     # ---------------- Graph clustering (optional) ----------------
#     def cluster_graph(self):
        
        
#         algo = self.cfg.graph_cluster_algorithm.lower()
#         if algo == "none":
#             logger.info("Graph clustering disabled (algorithm=none).")
#             return None

#         # --- Leiden via igraph (optional) ---
#         try:
#             import igraph as ig
#             import leidenalg
#             if algo == "leiden":
#                 logger.info("Running Leiden clustering via igraph + leidenalg.")
#                 g = ig.Graph.TupleList(self.kg.edges(), directed=True)
#                 partition = leidenalg.find_partition(g, leidenalg.RBConfigurationVertexPartition)
#                 clusters = [list(map(str, com)) for com in partition]
#                 logger.info(f"Leiden found {len(clusters)} clusters.")
#                 return clusters
#         except ImportError:
#             logger.warning("igraph or leidenalg not installed; skipping Leiden clustering.")

#         # --- NetworkX community fallback ---
#         if nx_community is not None and algo in ("louvain", "greedy", "labelprop"):
#             logger.info("Running NetworkX community clustering (fallback).")
#             communities = nx_community.greedy_modularity_communities(self.kg.to_undirected())
#             clusters = [list(c) for c in communities]
#             logger.info(f"NetworkX found {len(clusters)} communities.")
#             return clusters

#         logger.warning("No clustering library available or unsupported algorithm; skipping clustering.")
#         return None

#     # ---------------- Node embeddings (node2vec) ----------------
#     def embed_graph_nodes(self):
#         if Node2Vec is None:
#             logger.warning("node2vec not installed; skipping node embedding.")
#             return None
#         if self.kg is None or self.kg.number_of_nodes() == 0:
#             logger.warning("KG empty; run build_kg() first.")
#             return None
#         params = self.cfg.node2vec_params
#         logger.info("Running Node2Vec to generate node embeddings...")
#         # node2vec expects undirected or directed networkx graph
#         node2vec = Node2Vec(self.kg, dimensions=params.get("dimensions", 128),
#                             walk_length=params.get("walk_length", 40),
#                             num_walks=params.get("num_walks", 10),
#                             workers=2)
#         model = node2vec.fit(window=params.get("window_size", 3), min_count=1, batch_words=4)
#         # store embeddings in dict
#         self.node_embeddings = {node: model.wv[str(node)] for node in self.kg.nodes()}
#         logger.info(f"Generated node embeddings for {len(self.node_embeddings)} nodes.")
#         # optional cache
#         try:
#             np.save(os.path.join(self.cfg.working_dir, "node_embeddings.npy"), self.node_embeddings)
#         except Exception:
#             pass
#         return self.node_embeddings

#     # ---------------- GraphRAG Query (hybrid) ----------------
#     def _semantic_search(self, query: str, top_k: int = 5):
#         q_emb = self.embedding_model.encode([query], normalize_embeddings=True).astype("float32")
#         D, I = self.index.search(q_emb, top_k)
#         return [(int(i), float(D[0][idx])) for idx, i in enumerate(I[0])]

#     def _graph_search_scores(self, query: str):
#         q_doc = self.nlp(query)
#         q_entities = set([ent.text.strip() for ent in q_doc.ents if ent.text.strip()])
#         if len(q_entities) == 0:
#             return np.zeros(len(self.dataset), dtype=float)
#         scores = np.array([len(self.chunk_entities[i].intersection(q_entities)) for i in range(len(self.dataset))], dtype=float)
#         if scores.max() > 0:
#             scores = scores / (scores.max() + 1e-12)
#         return scores

#     def graphrag_query(self, query: str, top_k: int = 5, alpha: float = 0.8):
#         """
#         Combine semantic + graph signals.
#         alpha: weight for semantic retrieval (0..1). final_score = alpha*sem + (1-alpha)*graph
#         """
#         # semantic candidates
#         sem = self._semantic_search(query, top_k=top_k * 2)  # retrieve more to allow reranking
#         sem_idxs = [i for i, s in sem]
#         sem_scores = {i: s for (i, s) in sem}

#         # graph scores across dataset
#         graph_scores = self._graph_search_scores(query)

#         # compute combined score for sem candidates
#         combined = []
#         for i, sem_score in sem:
#             gscore = graph_scores[i] if i < len(graph_scores) else 0.0
#             score = alpha * sem_score + (1.0 - alpha) * gscore
#             combined.append((i, score))

#         # sort and pick top_k
#         combined_sorted = sorted(combined, key=lambda x: x[1], reverse=True)[:top_k]
#         combined_idxs = [i for i, s in combined_sorted]
#         combined_chunks = [self.dataset[i] for i in combined_idxs]

#         # gather also top graph-only chunks (if graph points to others not in sem candidates)
#         top_graph_idx = np.argsort(graph_scores)[-top_k:][::-1]
#         graph_chunks = [self.dataset[i] for i in top_graph_idx if graph_scores[i] > 0 and i not in combined_idxs]

#         # KG facts extraction (neighbours of query entities)
#         kg_facts = []
#         q_doc = self.nlp(query)
#         q_entities = set([ent.text.strip() for ent in q_doc.ents if ent.text.strip()])
#         for qe in q_entities:
#             if self.kg is not None and qe in self.kg:
#                 # successors & predecessors (limit to a few)
#                 for nb in list(self.kg.successors(qe))[:3]:
#                     rel = self.kg.get_edge_data(qe, nb).get("relation", "related_to")
#                     kg_facts.append(f"{qe} {rel} {nb}.")
#                 for nb in list(self.kg.predecessors(qe))[:3]:
#                     rel = self.kg.get_edge_data(nb, qe).get("relation", "related_to")
#                     kg_facts.append(f"{nb} {rel} {qe}.")

#         combined_context = " ".join(combined_chunks + graph_chunks + kg_facts)
#         # combined_context = " ".join(combined_chunks[:3] + kg_facts[:4])
            
#         return {"query": query, "context": combined_context, "sem_idxs": combined_idxs}
    
    
#     # ---------------- LLM Answer ----------------
#     def answer_question_llm(
#         self,
#         query: str,
#         llm_model: str = "moonshotai/kimi-k2-instruct-0905",
#         top_k: int = 5,
#         alpha: float = 0.8
#     ):
#         # result = self.graphrag_query(query, top_k=top_k, alpha=alpha)
#         # context_text = result["context"]
#         # 1) GraphRAG retrieve
#         result = self.graphrag_query(query, top_k=top_k, alpha=alpha)
#         context_text = result["context"]

#         # 2) QWEN reasoning (local, SAT-trained)
#         reasoning = qwen_reasoning(query, context_text)

#         final_prompt = f"""
# You are a high-accuracy QA assistant. 

# You MUST answer using ONLY the provided context.

# Your output must follow these rules:

# -------------------------------------------------
# DETECTION RULES (auto-detect the question type):
# - If the question asks for a specific name, place, person, object, event,
#   → classify as: FACT RETRIEVAL.
# - If the question asks “why”, “how”, “what happened”, “what was the relationship”,
#   or requires combining multiple clues,
#   → classify as: COMPLEX REASONING.
# -------------------------------------------------

# ANSWER STYLE RULES:
# 1. FACT RETRIEVAL:
#    - Answer in **3–12 words**.
#    - Must be short, factual, and contain ONLY the requested entity.
#    - Do NOT add explanations or context.

# 2. COMPLEX REASONING:
#    - Answer in 1–2 concise sentences (12–25 words total).
#    - Combine only the essential evidence from the context.
#    - No paraphrasing of long passages; no extra narrative.
#    - The answer must be strictly grounded in the provided context.

# 3. If the question_type is "Contextual Summarize":
#    - Produce 2–4 sentences.
#    - Length must be 80–200 words.
#    - Provide a smooth, coherent summary combining *all* major points from context.
#    - Use descriptive but concise language.
#    - Do NOT shorten aggressively.
#    - Do NOT reduce to a single-sentence summary.
#    - This rule overrides any sentence-limit rules.

# 4. If the question_type is "Creative Generation":
#    - Follow the exact requested format (diary entry, newspaper article, retelling, etc.).
#    - Stay grounded in the given evidence; do not introduce new events or characters.
#    - 3–6 sentences depending on style, expressive but controlled.

# 5. GENERAL RULES:
#     - Do NOT add information not present in the context.
#     - If context is incomplete, give the best-supported answer.
#     - If no relevant information exists, reply EXACTLY:
#     no relevant information found

# -------------------------------------------------
# CONTEXT:
# {context_text}

# QUESTION:
# {query}

# SAT REASONING:
# {reasoning}

# ANSWER:
# """
#         # fix: chạy async function trong sync
#         # import asyncio
#         # answer = asyncio.run(llm_generate(prompt, model=llm_model))
#         answer = llm_generate(final_prompt, model=llm_model)

#         return answer, result
       





#     # ---------------- Evaluation ----------------
#     def evaluate_answer(self, pred: str, truth: str) -> float:
#         scorer = Rouge()
#         scores = scorer.get_scores(pred, truth)
#         return scores[0]["rouge-l"]["f"]

#     # ---------------- Cache helpers ----------------
#     def save_index(self, fname: str = "faiss.index"):
#         if self.index is None:
#             raise ValueError("No index to save.")
#         path = os.path.join(self.cfg.working_dir, fname)
#         faiss.write_index(self.index, path)
#         logger.info(f"FAISS index written to {path}")

#     def load_index(self, fname: str = "faiss.index"):
#         path = os.path.join(self.cfg.working_dir, fname)
#         if not os.path.exists(path):
#             raise FileNotFoundError(path)
#         self.index = faiss.read_index(path)
#         logger.info(f"FAISS index loaded from {path}")

#     def save_embeddings(self, fname: str = "embeddings.npy"):
#         if self.embeddings is None:
#             raise ValueError("No embeddings to save.")
#         np.save(os.path.join(self.cfg.working_dir, fname), self.embeddings)
#         logger.info("Embeddings saved.")

#     def load_embeddings(self, fname: str = "embeddings.npy"):
#         path = os.path.join(self.cfg.working_dir, fname)
#         if not os.path.exists(path):
#             raise FileNotFoundError(path)
#         self.embeddings = np.load(path)
#         logger.info("Embeddings loaded.")

#     # ---------------- Convenience run -> inference on JSON questions ----------------
# def run_inference_on_medical_json(rag, json_path, num_questions=10,question_type_filter="Fact Retrieval"):
#     """
#     Chạy LLM trên các câu hỏi JSON được lọc theo question_type.
#     Mặc định chạy các câu 'Fact Retrieval'.
#     """
#     with open(json_path, "r", encoding="utf-8") as f:
#         data = json.load(f)
#     # Lọc các câu hỏi theo loại yêu cầu
#     # Lọc các câu hỏi theo loại yêu cầu
#     filtered = [
#         item for item in data
#         if item.get("question_type", "").lower() == question_type_filter.lower()
#     ]

#     if not filtered:
#         print(f"❌ Không tìm thấy câu hỏi loại '{question_type_filter}' trong file: {json_path}")
#         return

#     print(f"🔹 Found {len(filtered)} questions of type '{question_type_filter}'")
#     print(f"➡ Running inference on first {num_questions}...\n")

#     # print(f"🔹 Running inference on {num_questions} questions from {json_path}\n")

#     # ❗ USE filtered list, NOT full dataset
#     for i, item in enumerate(filtered[:num_questions]):
#         question = item.get("question", "") or item.get("q", "")
#         ground_truth = item.get("answer", "") or item.get("a", "")
#         # question_type = item.get("question_type", "")
#         # gọi LLM
#         ans, _ = rag.answer_question_llm(question)

#         # xử lý câu trả lời
#         sentences = [s.strip() for s in ans.replace("\n", " ").split(".") if s.strip()]
#         short_ans = ". ".join(sentences[:5]).strip()
#         if short_ans and not short_ans.endswith("."):
#             short_ans += "."

#         # in kết quả
#         print(f"=== Q{i+1}: {question}")
#         print(f"Kimi K2 Answer:\n{short_ans}\n")
#         print(f"Ground Truth:\n{ground_truth}\n")
#         print("-" * 120)


# # ---------------- Example usage ----------------
# if __name__ == "__main__":
#     # create rag with default config (you can pass a GraphRAGConfig instance to override)
#     rag = SimpleGraphRAG(embedding_model_name="all-MiniLM-L6-v2", chunk_size=1200, chunk_overlap=100)

#     # Paths (change to your local dataset)
#     corpus_path = "/Users/duyanhle1501/Duy Anh Le Code/GraphRAG-Benchmark/Datasets/Corpus/medical.json"
#     questions_path = "/Users/duyanhle1501/Duy Anh Le Code/GraphRAG-Benchmark/Datasets/Questions/medical_questions.json"

#     # 1) load corpus
#     text = rag.load_json_and_concat(corpus_path)

#     # 2) chunk (uses config tokenizer; default is tiktoken if available)
#     rag.chunk_text(text, method="auto")

#     # 3) embedding + index
#     rag.build_embeddings_and_index()

#     # 4) build a proper KG (NER + simple dependency relations)
#     rag.build_kg(text, use_dependency=True)

#     # import pickle
#     # import os

#     # os.makedirs("sat_data", exist_ok=True)

#     # with open("sat_data/kg.pkl", "wb") as f:
#     #     pickle.dump(rag.kg, f)

#     # print("✅ Saved KG to sat_data/kg.pkl")
#     # print("Nodes:", rag.kg.number_of_nodes())
#     # print("Edges:", rag.kg.number_of_edges())


#     # 5) optional: cluster graph & embed nodes
#     clusters = rag.cluster_graph()
#     node_embs = rag.embed_graph_nodes()

#     # 6) run inference on first 5 questions and print top 5-sentence summary
#     run_inference_on_medical_json(rag, questions_path, num_questions=10, question_type_filter="Fact Retrieval")




Đo khi đã có cluster

In [ ]:
pip install sentence-transformers faiss-cpu networkx nest_asyncio

In [ ]:
import os
os.environ["NVAPI_KEY"] = "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"

In [ ]:
# 3) Create a client wrapper
from openai import OpenAI
client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key=os.getenv("NVAPI_KEY")
)



In [ ]:
# fast_graphrag_full.py
"""
Single-file FastGraphRAG (adapted).
- Chunking: FastGraphRAG-like token-based chunking (800 tokens ≈ 3200 chars).
- IE: LLM-first extraction (Kimi wrapper), fallback heuristics.
- Storage: minimal IGraphStorage (networkx-backed).
- Embedding: sentence-transformers (optional), FAISS index (optional).
- Main: insert corpus and answer first 5 questions from a JSON file.

USAGE:
  - Put your NVAPI key into an environment variable NVAPI_KEY in the notebook, e.g.:
      import os
      os.environ["NVAPI_KEY"] = "nvapi-...."
  - Provide a small async wrapper that calls your client and pass to KimiLLMService.
  - Run the notebook cell to execute main.

Do NOT hardcode API keys in this file.
"""

import os
import json
import re
import logging
import asyncio
from dataclasses import dataclass, field
from typing import Any, Callable, Dict, List, Optional, Tuple, Iterable, Awaitable

import numpy as np
import networkx as nx
from itertools import chain

# Optional deps (guarded)
try:
    import faiss
except Exception:
    faiss = None
try:
    from sentence_transformers import SentenceTransformer
except Exception:
    SentenceTransformer = None
try:
    import nest_asyncio
except Exception:
    nest_asyncio = None

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("FastGraphRAG_full")

# ----------------------------
# Helpers
# ----------------------------

def run_async(coro):
    """Run coroutine safely both in notebooks and scripts."""
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            if nest_asyncio is not None:
                nest_asyncio.apply()
            return loop.run_until_complete(coro)
        else:
            return loop.run_until_complete(coro)
    except RuntimeError:
        return asyncio.run(coro)

# ----------------------------
# Constants
# ----------------------------
TOKEN_TO_CHAR_RATIO = 4  # approximate: 1 token ≈ 4 characters

# ----------------------------
# Embedding service (optional)
# ----------------------------
class BaseEmbeddingService:
    async def aembed_documents(self, texts: List[str]) -> np.ndarray:
        raise NotImplementedError

    async def aembed_query(self, text: str) -> np.ndarray:
        raise NotImplementedError

class SentenceEmbeddingService(BaseEmbeddingService):
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        if SentenceTransformer is None:
            raise RuntimeError("Please install sentence-transformers to use SentenceEmbeddingService.")
        self.model = SentenceTransformer(model_name)
        self.embedding_dim = self.model.get_sentence_embedding_dimension()

    async def aembed_documents(self, texts: List[str]) -> np.ndarray:
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(None, lambda: np.array(self.model.encode(texts, convert_to_numpy=True)))

    async def aembed_query(self, text: str) -> np.ndarray:
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(None, lambda: np.array(self.model.encode([text], convert_to_numpy=True)[0]))

# ----------------------------
# Kimi LLM wrapper (user provides completion_func)
# ----------------------------
class KimiLLMService:
    """
    Wrap an async completion callable:
      async def completion_func(prompt: str, **kwargs) -> str
    The wrapper simply forwards prompt and returns the text response.
    """
    def __init__(self, completion_func: Callable[[str], Awaitable[str]], model_name: str = "moonshotai/kimi-k2-instruct-0905"):
        self.completion_func = completion_func
        self.model = model_name

    async def acomplete(self, prompt: str, **kwargs) -> str:
        return await self.completion_func(prompt, **kwargs)

# ----------------------------
# FastGraphRAG config
# ----------------------------
@dataclass
class FastGraphRAGConfig:
    working_dir: str = field(default_factory=lambda: f"./fast_graphrag_cache_{np.random.randint(1e6)}")
    embedding_model_name: str = "all-MiniLM-L6-v2"
    llm_model_name: str = "moonshotai/kimi-k2-instruct-0905"
    chunk_token_size: int = 800
    chunk_token_overlap: int = 100
    # The above token-based values are converted to char approximations using TOKEN_TO_CHAR_RATIO.

# ----------------------------
# Chunking service (FastGraphRAG-like)
# ----------------------------
DEFAULT_SEPARATORS = ["\n\n\n", "\n\n", "\r\n\r\n", "。", "．", ".", "！", "!", "？", "?"]

@dataclass
class DefaultChunkingServiceConfig:
    separators: List[str] = field(default_factory=lambda: DEFAULT_SEPARATORS)
    chunk_token_size: int = 500
    chunk_token_overlap: int = 100

class DefaultChunkingService:
    def __init__(self, cfg: Optional[DefaultChunkingServiceConfig] = None):
        self.config = cfg or DefaultChunkingServiceConfig()
        self._split_re = re.compile(f"({'|'.join(re.escape(s) for s in self.config.separators or [])})")
        self._chunk_size = int(self.config.chunk_token_size * TOKEN_TO_CHAR_RATIO)
        self._chunk_overlap = int(self.config.chunk_token_overlap * TOKEN_TO_CHAR_RATIO)

    async def extract(self, documents: Iterable[str]) -> List[List[Dict[str, Any]]]:
        """Return list of list-of-chunks: [{'id': int, 'content': str, 'metadata': {}} ...]"""
        out = []
        for doc in documents:
            chunks = await self._extract_chunks(doc)
            seen = set()
            uniq = []
            for chunk in chunks:
                # stable hash: use Python hash salted by content -> absolute into 63-bit
                hid = abs(hash(chunk)) % (2**63)
                if hid not in seen:
                    seen.add(hid)
                    uniq.append({"id": hid, "content": chunk, "metadata": {}})
            out.append(uniq)
        return out

    async def _extract_chunks(self, text: str) -> List[str]:
        if text is None:
            return []
        # sanitize
        try:
            text = text.encode(errors="replace").decode()
        except Exception:
            text = re.sub(r"[\x00-\x09\x11-\x12\x14-\x1f]", " ", text)

        if len(text) <= self._chunk_size:
            return [text]

        parts = self._split_re.split(text)
        parts.append("")  # sentinel
        merged: List[List[str]] = []
        cur: List[str] = []
        cur_len = 0
        for i, p in enumerate(parts):
            plen = len(p)
            is_sep = (i % 2 == 1)
            if is_sep or (cur_len + plen <= self._chunk_size - (self._chunk_overlap if len(merged) > 0 else 0)):
                cur.append(p)
                cur_len += plen
            else:
                merged.append(cur)
                cur = [p]
                cur_len = plen
        if cur:
            merged.append(cur)

        # enforce overlap
        out: List[str] = []
        for idx, chunk_parts in enumerate(merged):
            if idx == 0:
                out.append("".join(chunk_parts))
            else:
                prev = merged[idx - 1]
                ov = []
                ov_len = 0
                for part in reversed(prev):
                    if ov_len + len(part) > self._chunk_overlap:
                        break
                    ov.insert(0, part)
                    ov_len += len(part)
                out.append("".join(ov + chunk_parts))
        return out

# ----------------------------
# Minimal IGraphStorage (networkx backed) — simplified API
# ----------------------------
class IGraphStorage:
    def __init__(self):
        self._G = nx.DiGraph()
        self._node_list: List[str] = []
        self._edge_list: List[Tuple[str, str]] = []
        self._in_progress = False

    async def insert_start(self):
        self._in_progress = True

    async def insert_done(self):
        # refresh lists for stable indexing
        self._node_list = list(self._G.nodes())
        self._edge_list = [(u, v) for u, v in self._G.edges()]
        self._in_progress = False

    async def node_count(self) -> int:
        return self._G.number_of_nodes()

    async def edge_count(self) -> int:
        return self._G.number_of_edges()

    async def add_entity(self, name: str, attrs: Optional[Dict[str, Any]] = None) -> int:
        attrs = attrs or {}
        if not self._G.has_node(name):
            self._G.add_node(name, **attrs)
        else:
            self._G.nodes[name].update(attrs)
        if name not in self._node_list:
            self._node_list.append(name)
        return self._node_list.index(name)

    async def add_relation(self, source: str, target: str, attrs: Optional[Dict[str, Any]] = None) -> int:
        attrs = attrs or {}
        if not self._G.has_node(source):
            await self.add_entity(source, {})
        if not self._G.has_node(target):
            await self.add_entity(target, {})
        self._G.add_edge(source, target, **attrs)
        self._edge_list.append((source, target))
        return len(self._edge_list) - 1

    async def get_node_by_index(self, idx: int) -> Optional[Dict[str, Any]]:
        try:
            n = self._node_list[idx]
            d = dict(self._G.nodes[n])
            d["_name"] = n
            return d
        except Exception:
            return None

    async def get_edge_by_index(self, idx: int) -> Optional[Dict[str, Any]]:
        try:
            s, t = self._edge_list[idx]
            attrs = dict(self._G[s][t])
            return {"source": s, "target": t, **attrs}
        except Exception:
            return None

    async def are_neighbours(self, src_idx: int, tgt_idx: int) -> bool:
        try:
            s = self._node_list[src_idx]; t = self._node_list[tgt_idx]
            return self._G.has_edge(s, t)
        except Exception:
            return False

    async def get_entities_to_relationships_map(self):
        rels = list(self._G.edges(data=True))
        ent_to_rel = []
        for node_idx, node in enumerate(self._node_list):
            l = []
            for rel_idx, (s, t, data) in enumerate(rels):
                if s == node or t == node:
                    l.append(rel_idx)
            ent_to_rel.append(l)
        return ent_to_rel

    async def get_relationships_attrs(self, key: str = "chunks"):
        rels = list(self._G.edges(data=True))
        return [data.get(key, []) for _, _, data in rels]

    async def save_graphml(self, output_path: str):
        nx.write_graphml(self._G, output_path)

# ----------------------------
# Information Extraction Service (LLM-first, 1-pass gleaning)
# ----------------------------
# ----------------------------
# Information Extraction Service (no JSON required)
# ----------------------------
@dataclass
class DefaultInformationExtractionService:
    llm: KimiLLMService
    # max_gleaning_steps không cần nữa, bỏ đi

    async def extract_from_chunk(self, chunk_text: str, chunk_id: int) -> Dict[str, List[Dict[str, Any]]]:
        """
        Simple entity/relationship extraction without requiring JSON.
        - Entities: capitalized phrases
        - Relationships: co-occurrence in sentence
        """
        ents = []
        seen = set()
        for m in re.finditer(r"\b([A-Z][a-z]{2,}(?:\s+[A-Z][a-z]{2,})*)\b", chunk_text):
            name = m.group(1).strip()
            if name.lower() in seen:
                continue
            seen.add(name.lower())
            ents.append({"name": name, "type": "UNKNOWN", "desc": ""})

        rels = []
        sents = re.split(r"[.!?]\s+", chunk_text)
        for sent in sents:
            found = [e["name"] for e in ents if e["name"] in sent]
            for i in range(len(found)):
                for j in range(i+1, len(found)):
                    rels.append({
                        "source": found[i],
                        "target": found[j],
                        "desc": "co_occurs_in_sentence",
                        "chunks": [chunk_id]
                    })

        return {"entities": ents, "relationships": rels}
    #     prompt = f"""
    # Extract important nouns and proper nouns from this text.
    # Return them as a comma-separated list.
    # TEXT: {chunk_text[:1500]}
    # """
    #     raw = await self.llm.acomplete(prompt)
    #     words = [w.strip() for w in raw.split(",") if w.strip()]

    #     ents = []
    #     for w in words:
    #         ents.append({"name": w, "type": "UNKNOWN", "desc": ""})

    #     # simple co-occurrence
    #     rels = []
    #     for i in range(len(words)):
    #         for j in range(i+1, len(words)):
    #             rels.append({
    #                 "source": words[i],
    #                 "target": words[j],
    #                 "desc": "co_occurs",
    #                 "chunks": [chunk_id]
    #             })
    #     return {"entities": ents, "relationships": rels}

    async def gleaning(self, partial_graph: Dict[str, Any]) -> Dict[str, Any]:
        """
        No LLM post-processing, just return partial_graph as-is
        """
        return partial_graph


# ----------------------------
# Main FastGraphRAG class
# ----------------------------
class FastGraphRAG:
    def __init__(self, cfg: Optional[FastGraphRAGConfig] = None, embedding_service: Optional[BaseEmbeddingService] = None, llm_service: Optional[KimiLLMService] = None):
        self.cfg = cfg or FastGraphRAGConfig()
        os.makedirs(self.cfg.working_dir, exist_ok=True)

        # embedding
        if embedding_service is None:
            if SentenceTransformer is not None:
                self.embedding_service = SentenceEmbeddingService(self.cfg.embedding_model_name)
            else:
                self.embedding_service = None
        else:
            self.embedding_service = embedding_service

        # llm
        if llm_service is None:
            raise RuntimeError("Please provide a KimiLLMService instance (completion wrapper).")
        self.llm_service = llm_service

        # chunking & IE
        chunk_cfg = DefaultChunkingServiceConfig(chunk_token_size=self.cfg.chunk_token_size, chunk_token_overlap=self.cfg.chunk_token_overlap)
        self.chunking_service = DefaultChunkingService(chunk_cfg)
        # self.ie_service = DefaultInformationExtractionService(llm=self.llm_service, max_gleaning_steps=1)
        self.ie_service = DefaultInformationExtractionService(llm=self.llm_service)


        # storage & index
        self.kg = IGraphStorage()
        self.chunks: List[str] = []
        self.chunk_ids: List[int] = []
        self.embeddings = None
        self.index = None

    async def async_insert(self, content: Any):
        # load text
        text = ""
        if isinstance(content, str) and os.path.exists(content):
            with open(content, "r", encoding="utf-8") as f:
                data = json.load(f)
            if isinstance(data, list):
                text = "\n".join([i.get("text") or i.get("context") or "" for i in data])
            elif isinstance(data, dict):
                text = data.get("text") or data.get("context") or " ".join(map(str, data.values()))
            else:
                text = str(data)
        else:
            text = content if isinstance(content, str) else " ".join(map(str, content))

        # chunk using service
        chunk_lists = await self.chunking_service.extract([text])
        chunks = chunk_lists[0]
        self.chunks = [c["content"] for c in chunks]
        self.chunk_ids = [c["id"] for c in chunks]

        # embeddings (if embedding service available)
        if self.embedding_service is not None:
            logger.info("Embedding %d chunks...", len(self.chunks))
            embs = await self.embedding_service.aembed_documents(self.chunks)
            embs = embs / (np.linalg.norm(embs, axis=1, keepdims=True) + 1e-9)
            self.embeddings = embs.astype("float32")
            if faiss is not None:
                import faiss as _faiss
                self.index = _faiss.IndexFlatIP(self.embeddings.shape[1])
                self.index.add(self.embeddings)
                logger.info("Built FAISS index with %d vectors", self.embeddings.shape[0])
            else:
                logger.warning("FAISS not installed; retrieval will fall back to substring search.")
                self.index = None
        else:
            logger.warning("No embedding service provided; retrieval will be substring-based.")
            self.index = None

        # information extraction & build KG
        await self.kg.insert_start()
        try:
            for i, chunk_text in enumerate(self.chunks):
                res = await self.ie_service.extract_from_chunk(chunk_text, chunk_id=self.chunk_ids[i])
                ents = res.get("entities", []) or []
                rels = res.get("relationships", []) or []

                for ent in ents:
                    name = (ent.get("name") or "").strip()
                    if not name:
                        continue
                    attrs = {"type": ent.get("type", ""), "description": ent.get("desc", "")}
                    await self.kg.add_entity(name, attrs)

                for rel in rels:
                    src = (rel.get("source") or "").strip()
                    tgt = (rel.get("target") or "").strip()
                    if not src or not tgt:
                        continue
                    attrs = {"description": rel.get("desc", ""), "chunks": rel.get("chunks", [self.chunk_ids[i]])}
                    await self.kg.add_relation(src, tgt, attrs)
        finally:
            await self.kg.insert_done()

        # one gleaning pass across the whole KG
        partial = {"entities": [], "relationships": []}
        for n in self.kg._node_list:
            data = dict(self.kg._G.nodes[n])
            partial["entities"].append({"name": n, "type": data.get("type", ""), "desc": data.get("description", "")})
        for (s, t, data) in self.kg._G.edges(data=True):
            partial["relationships"].append({"source": s, "target": t, "desc": data.get("description", ""), "chunks": data.get("chunks", [])})

        updated = await self.ie_service.gleaning(partial)
        new_ents = updated.get("entities", [])
        new_rels = updated.get("relationships", [])
        for ent in new_ents:
            name = (ent.get("name") or "").strip()
            if name and name not in self.kg._G.nodes:
                await self.kg.add_entity(name, {"type": ent.get("type", ""), "description": ent.get("desc", "")})
        for rel in new_rels:
            s = (rel.get("source") or "").strip()
            t = (rel.get("target") or "").strip()
            if s and t and not self.kg._G.has_edge(s, t):
                await self.kg.add_relation(s, t, {"description": rel.get("desc", ""), "chunks": rel.get("chunks", [])})

        return len(self.chunks)

    def insert(self, content: Any):
        return run_async(self.async_insert(content))

    # async def async_query(self, query: str, top_k: int = 5):
    #     # retrieval
    #     if self.index is not None and self.embedding_service is not None:
    #         q_emb = await self.embedding_service.aembed_query(query)
    #         D, I = self.index.search(np.array([q_emb], dtype="float32"), top_k)
    #         ctx = " ".join([self.chunks[i] for i in I[0] if i < len(self.chunks)])
    #     else:
    #         # substring fallback: pick top_k chunks that contain most query tokens (simple heuristic)
    #         tokens = re.findall(r"\w+", query.lower())
    #         scores = []
    #         for c in self.chunks:
    #             cnt = sum(1 for t in tokens if t in c.lower())
    #             scores.append(cnt)
    #         idxs = sorted(range(len(self.chunks)), key=lambda i: scores[i], reverse=True)[:top_k]
    #         ctx = " ".join([self.chunks[i] for i in idxs])

    #     # prompt = (
    #     #     "You are a helpful assistant. Use ONLY the context below to answer the question. "
    #     #     "If answer cannot be found, reply: \"no relevant information found\".\n\n"
    #     #     f"CONTEXT:\n{ctx}\n\nQUESTION:\n{query}\n\n"
    #     # )
    #     prompt = (
    #         "You are a concise QA assistant.\n"
    # "Your task is to answer the question using ONLY the provided context.\n\n"
    # "RULES:\n"
    # "- The answer MUST be extremely short.\n"
    # "- Prefer a single word or a short noun phrase (1–3 words).\n"
    # "- Do NOT write full sentences.\n"
    # "- Do NOT add explanations.\n"
    # "- If the context suggests an answer, output ONLY the answer.\n"
    # "- If the answer is a name, output ONLY the name.\n"
    # "- If no answer exists in the context, reply exactly: no relevant information found.\n\n"
    # f"CONTEXT:\n{ctx}\n\n"
    # f"QUESTION:\n{query}\n\n"
    # "ANSWER:"
    #     )

    #     ans = await self.llm_service.acomplete(prompt)
    #     return {"response": ans, "context": ctx}

    
    async def async_query(self, query: str, top_k: int = 10):
    # -----------------------------
    # 1) SEMANTIC RETRIEVAL
    # -----------------------------
        semantic_chunks = []
        if self.index is not None and self.embedding_service is not None:
            q_emb = await self.embedding_service.aembed_query(query)
            D, I = self.index.search(np.array([q_emb], dtype="float32"), top_k)
            semantic_chunks = [self.chunks[i] for i in I[0] if i < len(self.chunks)]

        # -----------------------------
        # 2) KEYWORD FALLBACK RETRIEVAL
        # -----------------------------
        tokens = re.findall(r"\w+", query.lower())
        keyword_scores = []
        for idx, c in enumerate(self.chunks):
            score = sum(1 for t in tokens if t in c.lower())
            keyword_scores.append((score, idx))

        keyword_scores.sort(reverse=True)
        keyword_chunks = [self.chunks[idx] for score, idx in keyword_scores[:top_k]]

        # -----------------------------
        # 3) MERGE + DEDUPLICATE CONTEXT
        # -----------------------------
        merged = semantic_chunks + keyword_chunks
        ctx_list = []
        seen = set()

        for x in merged:
            if x not in seen and len(x.strip()) > 0:
                ctx_list.append(x)
                seen.add(x)

        ctx = "\n\n".join(ctx_list)

        if not ctx.strip():
            ctx = "\n\n".join(self.chunks[:min(3, len(self.chunks))])

        # -----------------------------
        # 4) ADAPTIVE QA PROMPT
        # -----------------------------
            # You are a high-quality QA assistant. 
    # You MUST answer using ONLY the provided context.

    # ADAPTIVE ANSWER RULES:
    # 1. If the answer is a simple fact (e.g., name, person, location, plant, medical term),
    # respond in 3–12 words (not too short, not too long).

    # 2. If the question requires reasoning, combining clues, or narrative interpretation,
    # respond in **1–3 concise sentences**.

    # 3. Never invent information. Never add details not present in the context.

    # 4. If the context contains partial information, provide the best supported answer.

    # 5. Only reply exactly: no relevant information found
    # if the context truly contains zero clues.
        prompt = f"""
    
    You are a high-accuracy QA assistant. 

You MUST answer using ONLY the provided context.

Your output must follow these rules:

-------------------------------------------------
DETECTION RULES (auto-detect the question type):
- If the question asks for a specific name, place, person, object, event,
  → classify as: FACT RETRIEVAL.
- If the question asks “why”, “how”, “what happened”, “what was the relationship”,
  or requires combining multiple clues,
  → classify as: COMPLEX REASONING.
-------------------------------------------------

ANSWER STYLE RULES:
1. FACT RETRIEVAL:
   - Answer in **3–12 words**.
   - Must be short, factual, and contain ONLY the requested entity.
   - Do NOT add explanations or context.

2. COMPLEX REASONING:
   - Answer in 1–2 concise sentences (12–25 words total).
   - Combine only the essential evidence from the context.
   - No paraphrasing of long passages; no extra narrative.
   - The answer must be strictly grounded in the provided context.

3. If the question_type is "Contextual Summarize":
   - Produce 2–4 sentences.
   - Length must be 80–200 words.
   - Provide a smooth, coherent summary combining *all* major points from context.
   - Use descriptive but concise language.
   - Do NOT shorten aggressively.
   - Do NOT reduce to a single-sentence summary.
   - This rule overrides any sentence-limit rules.

4. If the question_type is "Creative Generation":
   - Follow the exact requested format (diary entry, newspaper article, retelling, etc.).
   - Stay grounded in the given evidence; do not introduce new events or characters.
   - 3–6 sentences depending on style, expressive but controlled.

5. GENERAL RULES:
    - Do NOT add information not present in the context.
    - If context is incomplete, give the best-supported answer.
    - If no relevant information exists, reply EXACTLY:
    no relevant information found

-------------------------------------------------

    CONTEXT:
    {ctx}

    QUESTION:
    {query}

    ANSWER:
    """

        # -----------------------------
        # 5) LLM Call
        # -----------------------------
        ans = await self.llm_service.acomplete(prompt)
        if ans:
            ans = ans.strip()

        # -----------------------------
        # 6) ADAPTIVE POST-PROCESSING
        # -----------------------------
        if ans:
            # Nếu câu quá dài (> 120 từ) → rút xuống 3 câu
            if len(ans.split()) > 120:
                sentences = re.split(r"[.!?]", ans)
                ans = ". ".join(sentences[:3]).strip()

            # Nếu mô hình trả lời 1 từ quá ngắn cho fact retrieval
            # → KHÔNG cắt lại (vì prompt đã đảm bảo 3–12 words)
            # Nên không làm gì ở đây

        return {"response": ans, "context": ctx}



    def query(self, query: str, top_k: int = 10):
        return run_async(self.async_query(query, top_k))

# ----------------------------
# Demo runner & main
# ----------------------------
def run_inference_on_json(rag: FastGraphRAG, json_path: str, num_questions: int = 20):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    for i, item in enumerate(data[:num_questions]):
        q = item.get("question", "") or item.get("query", "")
        gt = item.get("answer", "") or item.get("gt", "") or ""
        print(f"QUESTION {i+1} ")
        res = rag.query(q, top_k=5)
        print("LLM ANSWER:", res["response"])
        print("\nGROUND TRUTH:", gt)
        # print("-------------------------\n")

if __name__ == "__main__":
    # ---------- USER ACTIONS ----------
    # 1) Install dependencies in your notebook env:
    #    pip install sentence-transformers faiss-cpu networkx nest_asyncio
    #    (faiss-cpu optional; if absent you'll get substring fallback retrieval)
    #
    # 2) Set environment variable NVAPI_KEY in the notebook BEFORE running this file:
    #    import os
    #    os.environ["NVAPI_KEY"] = "nvapi-...."   # <--- put your key here (do NOT commit)
    #
    # 3) Create a client wrapper (example below uses the OpenAI-like client you mentioned)
    #
    # ---------- wiring example ----------
    from openai import OpenAI  # or the client SDK you have installed that matches the snippet
    # Do NOT hardcode key in the file; get it from env
    client = OpenAI(base_url="https://integrate.api.nvidia.com/v1", api_key=os.getenv("NVAPI_KEY"))

    # async wrapper around the sync client call (runs in executor)
    async def kimi_completion(prompt: str, **kwargs) -> str:
        def call():
            # Use your client to create a chat completion - adjust to your client's response shape
            resp = client.chat.completions.create(
                model="moonshotai/kimi-k2-instruct-0905",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=1024,
                temperature=0.6,
                top_p=0.9,
            )
            # adapt to your SDK's return format; this assumes resp.choices[0].message.content
            try:
                return resp.choices[0].message.content
            except Exception:
                # fallback: str(resp)
                return str(resp)
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(None, call)

    # Create LLM wrapper
    kimi_service = KimiLLMService(kimi_completion)

    # Create embedding service (optional)
    if SentenceTransformer is not None:
        emb_service = SentenceEmbeddingService("all-MiniLM-L6-v2")
    else:
        emb_service = None

    # Instantiate RAG
    cfg = FastGraphRAGConfig()
    rag = FastGraphRAG(cfg=cfg, embedding_service=emb_service, llm_service=kimi_service)

    # Paths (update to your dataset paths)
    corpus_path = "/Users/duyanhle1501/Duy Anh Le Code/GraphRAG-Benchmark/Datasets/Corpus/novel.json"
    questions_path = "/Users/duyanhle1501/Duy Anh Le Code/GraphRAG-Benchmark/Datasets/Questions/novel_questions.json"

    print("Inserting corpus (this may take some time)...")
    rag.insert(corpus_path)
    print("Done inserting.")

    print("Running first 5 questions from questions file...")
    run_inference_on_json(rag, questions_path, num_questions=20)

